# 🧠 om-memory: Observational Memory for AI Agents

**Human-like memory for AI agents. 10x cheaper than RAG for conversations. Zero vector DB needed.**

---

## The Problem

Traditional RAG systems are great for **knowledge retrieval**, but they fail at **conversational memory**:

| Turn | Traditional RAG Context | om-memory Context |
|------|------------------------|-------------------|
| 1 | System + Docs + Q1 (~800 tokens) | System + Docs + Q1 (~800 tokens) |
| 5 | System + Docs + **full history** (~3,500 tokens) | System + Docs + **memory** (~1,000 tokens) |
| 10 | System + Docs + **full history** (~8,000 tokens) 😱 | System + Docs + **memory** (~1,200 tokens) 🎉 |
| 50 | System + Docs + **full history** (~40,000 tokens) 💸 | System + Docs + **memory** (~1,500 tokens) ✅ |

**om-memory** observes conversations like a human — extracting key facts, decisions, and preferences into structured observations, then discarding the raw chat history.

### Key Features
- 🔴🟡🟢 **Priority-based observations** — Critical facts vs nice-to-know details
- 🧵 **Thread-scoped memory** — Per-conversation context
- 👤 **Resource-scoped memory** — Shared across a user's threads
- 🔄 **Rolling window** — Keeps recent messages, compresses older ones
- 📊 **Built-in observability** — Token tracking, cost savings metrics
- 🧪 **Demo mode** — Low thresholds for testing

---

## 📦 Installation

In [ ]:
!pip install -q "om-memory[all]==0.2.2" nest-asyncio

## 🔑 API Key Setup

In [ ]:
import os
import nest_asyncio

nest_asyncio.apply()

# Enter your OpenAI API key here
os.environ["OPENAI_API_KEY"] = "sk-..."  # <-- Replace with your key

print("✅ API key configured.")

---
## 1️⃣ Basic Usage: Add Messages & Get Context

The simplest flow — add messages to a thread, then retrieve the compressed context.

In [ ]:
import asyncio
from om_memory import ObservationalMemory

# Initialize with demo mode (low thresholds for testing)
om = ObservationalMemory()
om.config.demo_mode = True
om.config.observer_token_threshold = 200  # Very low for demo

async def basic_demo():
    thread_id = "basic_demo_01"
    
    # Add some conversation messages
    messages = [
        ("user", "Hi, I'm Sarah. I'm a senior data scientist at the ML team."),
        ("assistant", "Hello Sarah! Great to meet you. How can I help you today?"),
        ("user", "I need to plan my vacation. I've used 8 PTO days this year already."),
        ("assistant", "You have 12 PTO days remaining (20 total - 8 used). Would you like help planning?"),
        ("user", "Yes! I want to go to Japan for 2 weeks in November."),
        ("assistant", "A 2-week trip to Japan in November sounds amazing! That would use 10 of your remaining 12 days."),
        ("user", "Perfect. Also, what's the remote work policy? I might want to work remotely for a few days before the trip."),
        ("assistant", "Senior employees can work remotely up to 3 days per week with manager approval."),
    ]
    
    for role, content in messages:
        await om.aadd_message(thread_id, role, content)
    
    # Get context BEFORE observation
    ctx_before = await om.aget_context(thread_id)
    print("=== CONTEXT BEFORE OBSERVATION ===")
    print(ctx_before)
    print(f"\n(~{len(ctx_before)//4} tokens)")
    
    # Force observation — compress the conversation
    print("\n\n🔄 Running observation (compressing conversation)...\n")
    await om.aobserve(thread_id)
    
    # Get context AFTER observation
    ctx_after = await om.aget_context(thread_id)
    print("=== CONTEXT AFTER OBSERVATION ===")
    print(ctx_after)
    print(f"\n(~{len(ctx_after)//4} tokens)")
    print(f"\n✅ Compression: {len(ctx_before)//4} → {len(ctx_after)//4} tokens ({(1 - len(ctx_after)/len(ctx_before))*100:.0f}% smaller)")

asyncio.run(basic_demo())

---
## 2️⃣ Resource-Scoped Memory: Cross-Thread Sharing

Unlike traditional RAG, om-memory can share observations **across multiple conversations** for the same user.

In [ ]:
async def resource_scoped_demo():
    om2 = ObservationalMemory()
    om2.config.demo_mode = True
    om2.config.observer_token_threshold = 200
    
    user_id = "user_sarah_42"  # Resource ID = shared across all her threads
    
    # ---- Thread 1: HR Chat ----
    print("=== Thread 1: HR Chat ===")
    t1 = "hr_chat_sarah"
    await om2.aadd_message(t1, "user", "I'm Sarah, a senior data scientist. I need to plan 2 weeks of PTO for Japan in November.", resource_id=user_id)
    await om2.aadd_message(t1, "assistant", "Got it, Sarah! You have 12 PTO days remaining.", resource_id=user_id)
    await om2.aadd_message(t1, "user", "I've been at the company for 6 years, so I get extra days right?", resource_id=user_id)
    await om2.aadd_message(t1, "assistant", "Correct! With 6 years of tenure, you get 25 total PTO days.", resource_id=user_id)
    await om2.aobserve(t1, resource_id=user_id)
    
    # ---- Thread 2: IT Support ----
    print("\n=== Thread 2: IT Support (DIFFERENT conversation) ===")
    t2 = "it_support_sarah"
    # This thread automatically gets Sarah's observations from Thread 1!
    ctx = await om2.aget_context(t2, resource_id=user_id)
    print("Context in Thread 2 (includes memory from Thread 1):")
    print(ctx)
    print("\n✅ The IT support thread already knows Sarah is a senior data scientist planning a Japan trip!")

asyncio.run(resource_scoped_demo())

---
## 3️⃣ Rolling Window: Message Retention

After observation, om-memory keeps a configurable number of recent messages (rolling window) instead of deleting everything.

In [ ]:
async def rolling_window_demo():
    om3 = ObservationalMemory()
    om3.config.demo_mode = True
    om3.config.observer_token_threshold = 100
    om3.config.message_retention_count = 2  # Keep only last 2 messages after observation
    
    thread_id = "rolling_demo"
    
    # Add 6 messages
    for i in range(1, 7):
        await om3.aadd_message(thread_id, "user", f"Message number {i} from user")
        await om3.aadd_message(thread_id, "assistant", f"Response number {i} from assistant")
    
    print(f"Messages before observation: {len(await om3.storage.aget_messages(thread_id))}")
    
    # Observe — this compresses and retains only the last 2 messages
    await om3.aobserve(thread_id)
    
    remaining = await om3.storage.aget_messages(thread_id)
    print(f"Messages after observation:  {len(remaining)}")
    print(f"\nRetained messages:")
    for m in remaining:
        print(f"  {m.role}: {m.content}")
    
    print(f"\n✅ Only kept the last {om3.config.message_retention_count} messages. Everything else is compressed into observations.")

asyncio.run(rolling_window_demo())

---
## 4️⃣ Viewing Observations: What om-memory Extracts

Let's see exactly what structured observations om-memory creates from raw conversations.

In [ ]:
async def observations_demo():
    om4 = ObservationalMemory()
    om4.config.demo_mode = True
    om4.config.observer_token_threshold = 150
    
    thread_id = "obs_demo"
    
    # A rich conversation with lots of extractable facts
    convos = [
        ("user", "I'm Marcus, an L4 Staff Engineer on the Platform team. Been here 8 years."),
        ("assistant", "Welcome Marcus! As an L4 Staff Engineer with 8 years of tenure, you're at a senior level."),
        ("user", "I'm considering going for L5 Principal Engineer. What do I need?"),
        ("assistant", "For L5 promotion, you need 10+ years experience and to set company-wide technical direction."),
        ("user", "I'm also doing on-call next week. The rotation schedule is alphabetical, right?"),
        ("assistant", "Yes, it rotates alphabetically. You'll get a $500 weekly stipend plus $100 per incident."),
        ("user", "Last thing — I'm presenting at KubeCon next month. What's the travel expense policy?"),
        ("assistant", "For conferences: $2,000 learning budget, $200/night hotel, $50/day meals, economy flights."),
    ]
    
    for role, content in convos:
        await om4.aadd_message(thread_id, role, content)
    
    await om4.aobserve(thread_id)
    
    observations = await om4.aget_observations(thread_id)
    
    print(f"📊 Extracted {len(observations)} observations from {len(convos)} messages:\n")
    for obs in observations:
        print(f"  {obs.priority.value} [{obs.observation_date.strftime('%H:%M')}] {obs.content}")
    
    print(f"\n✅ Key facts, decisions, and plans are preserved. Raw chat history is discarded.")

asyncio.run(observations_demo())

---
## 5️⃣ Cost Comparison: RAG vs om-memory

Let's quantify the actual token savings across a multi-turn conversation.

In [ ]:
async def cost_comparison():
    om5 = ObservationalMemory()
    om5.config.demo_mode = True
    om5.config.observer_token_threshold = 300
    om5.config.message_retention_count = 3
    
    thread_id = "cost_demo"
    
    messages = [
        "Hi, I'm Alex, a backend engineer.",
        "How many PTO days do I get?",
        "I've used 5 days. Planning a trip to Hawaii.",
        "What's the remote work policy for senior engineers?",
        "Tell me about the on-call rotation compensation.",
        "I'm going to re:Invent. What expenses are covered?",
        "Can I get business class for the flight? It's over 6 hours.",
        "What mental health benefits do we have?",
        "Remind me — how many PTO days did I say I've used?",
        "Thanks for all the info! One last thing — what's the 401k match?",
    ]
    
    # Track tokens for both approaches
    rag_tokens = []
    om_tokens = []
    full_history = ""
    
    for i, msg in enumerate(messages):
        # Traditional RAG: full history grows every turn
        full_history += f"user: {msg}\nassistant: [response]\n"
        rag_context_size = len(full_history) // 4
        rag_tokens.append(rag_context_size)
        
        # om-memory: compressed context stays flat
        await om5.aadd_message(thread_id, "user", msg)
        await om5.aadd_message(thread_id, "assistant", f"Response to: {msg[:50]}")
        
        ctx = await om5.aget_context(thread_id)
        om_context_size = len(ctx) // 4
        om_tokens.append(om_context_size)
    
    # Print comparison table
    print(f"{'Turn':<6} {'RAG Tokens':<15} {'om-memory Tokens':<18} {'Savings':<10}")
    print("-" * 50)
    for i in range(len(messages)):
        saving = ((rag_tokens[i] - om_tokens[i]) / max(rag_tokens[i], 1)) * 100
        print(f"{i+1:<6} {rag_tokens[i]:<15} {om_tokens[i]:<18} {saving:.0f}%")
    
    total_rag = sum(rag_tokens)
    total_om = sum(om_tokens)
    print(f"\n{'TOTAL':<6} {total_rag:<15} {total_om:<18} {((total_rag-total_om)/total_rag)*100:.0f}%")
    
    # Cost at gpt-4o-mini pricing ($0.15/1M tokens)
    rag_cost = total_rag * 0.15 / 1_000_000
    om_cost = total_om * 0.15 / 1_000_000
    print(f"\n💰 Estimated cost (gpt-4o-mini):")
    print(f"   RAG:       ${rag_cost:.6f}")
    print(f"   om-memory: ${om_cost:.6f}")
    print(f"   Saved:     ${rag_cost - om_cost:.6f}")
    print(f"\n📈 At 100K conversations/month: RAG=${rag_cost*100000:.2f} vs om-memory=${om_cost*100000:.2f}")

asyncio.run(cost_comparison())

---
## 6️⃣ Context Formats: Text, Dict, JSON

om-memory can output context in multiple formats for different integration patterns.

In [ ]:
import json

async def format_demo():
    om6 = ObservationalMemory()
    om6.config.demo_mode = True
    om6.config.observer_token_threshold = 100
    
    thread_id = "format_demo"
    await om6.aadd_message(thread_id, "user", "My name is Priya and I work on the ML team.")
    await om6.aadd_message(thread_id, "assistant", "Nice to meet you Priya!")
    await om6.aobserve(thread_id)
    
    # Text format
    print("=== TEXT FORMAT ===")
    print(await om6.aget_context(thread_id, format='text'))
    
    # Dict format
    print("\n=== DICT FORMAT ===")
    ctx_dict = await om6.aget_context(thread_id, format='dict')
    print(json.dumps(ctx_dict, indent=2, default=str))
    
    # JSON format
    print("\n=== JSON FORMAT ===")
    print(await om6.aget_context(thread_id, format='json'))

asyncio.run(format_demo())

---
## 🎯 Summary

| Feature | Description |
|---------|-------------|
| **Observational Memory** | Extracts key facts from conversations into structured observations |
| **Priority System** | 🔴 Critical, 🟡 Important, 🟢 Info — like human attention |
| **Rolling Window** | Keeps N recent messages, compresses the rest |
| **Resource-Scoped** | Share memory across multiple threads for the same user |
| **Demo Mode** | Low thresholds for testing and experimentation |
| **Token Savings** | 60-80% reduction in context window tokens vs full history |
| **Zero Vector DB** | No embedding infrastructure needed for memory (only for RAG knowledge) |

### Links
- 📦 [PyPI: om-memory](https://pypi.org/project/om-memory/)
- 🐙 [GitHub: pratik333/om-memory](https://github.com/pratik333/om-memory)
- 📖 [Documentation](https://github.com/pratik333/om-memory#readme)